# Максимизация R2

Метрика R2 понятнее RMSE. Остановимся на коэффициенте детерминации и подберём лучшую модель. Чтобы решить задачу успешно, доведем метрику до `0.14`.

In [1]:
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score

from joblib import dump

In [2]:
data = pd.read_csv('flights_preprocessed.csv')

RND_STATE = 12345

target = data['Arrival Delay']
features = data.drop(['Arrival Delay'] , axis=1)
features_train, features_valid, target_train, target_valid = train_test_split(
    features, target, test_size=0.25, random_state=RND_STATE)

Вычислим значение R2 функцией `score()` у модели. Это по умолчанию метрика для моделей регрессии в sklearn.

In [3]:
model = LinearRegression()
model.fit(features_train, target_train)
model.score(features_valid, target_valid)

0.09710508979020725

Подберем подходящую глубину дерева сначала при небольшом числе деревьев. С их увеличением качество модели всегда растёт. Но увеличивается и время обучения.

In [4]:
for depth in range(1, 16, 1):
    model = RandomForestRegressor(n_estimators=20, max_depth=depth, random_state=RND_STATE)
    model.fit(features_train, target_train)
    print(f"depth {depth}: {model.score(features_valid, target_valid)}")

depth 1: 0.037285907526148354
depth 2: 0.06819642209621124
depth 3: 0.08789931366785175
depth 4: 0.10419080408264147
depth 5: 0.11727849049064364
depth 6: 0.12888629498765314
depth 7: 0.14013669309593169
depth 8: 0.1501643078194318
depth 9: 0.15565993165328307
depth 10: 0.1574533216534546
depth 11: 0.154639053609258
depth 12: 0.15603402639125652
depth 13: 0.1522420314465106
depth 14: 0.146476463959043
depth 15: 0.14439352500511204


Теперь запустим обучение случайного леса с большим количеством деревьев:

In [15]:
%%time

model = RandomForestRegressor(n_estimators=100, max_depth=12, random_state=RND_STATE)
model.fit(features_train, target_train)
print("train:", model.score(features_train, target_train))
print("valid:", model.score(features_valid, target_valid))

train: 0.4447535263510586
valid: 0.16178917671554938
CPU times: user 18.5 s, sys: 91.6 ms, total: 18.6 s
Wall time: 16.7 s


In [16]:
dump(model, 'model.joblib')

['model.joblib']